In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import os
import sys
sys.path.append("/n/home12/binxuwang/Github/Closed-loop-visual-insilico")
import timm
import torch
import torch as th
import torch.nn as nn
from torchvision.models.feature_extraction import create_feature_extractor
from tqdm.auto import tqdm
from os.path import join
import pickle as pkl
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from horama import maco, plot_maco
import torchvision.transforms as T
from torchvision.transforms import ToPILImage, ToTensor, Normalize, Resize
from torchvision.models import resnet50
from circuit_toolkit.CNN_scorers import TorchScorer
from circuit_toolkit.GAN_utils import upconvGAN, Caffenet
from circuit_toolkit.plot_utils import to_imgrid, show_imgrid, save_imgrid, saveallforms
from circuit_toolkit.layer_hook_utils import featureFetcher_module, featureFetcher, get_module_names
from circuit_toolkit.dataset_utils import ImagePathDataset
from torch.utils.data import DataLoader

import sklearn
from sklearn.pipeline import make_pipeline
from sklearn.random_projection import SparseRandomProjection, GaussianRandomProjection
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV
from sklearn.decomposition import PCA
from sklearn.kernel_ridge import KernelRidge

#%% Utility Functions
from neural_regress.regress_lib import sweep_regressors
from neural_regress.sklearn_torchify_lib import SRP_torch, PCA_torch, LinearRegression_torch, SpatialAvg_torch
from core.data_utils import load_from_hdf5, load_neural_data, load_neural_trial_resp_tensor, create_response_tensor, parse_image_fullpaths

In [3]:
dataroot = r"/n/holylabs/LABS/alvarez_lab/Lab/VVS_Accentuation/Ephys_Data"
data_path = join(dataroot, "red_20250428-20240429_vvs-encodingstimuli_z1_rw100-400.h5")
data = load_from_hdf5(data_path)
print(list(data))

['README', 'neuron_metadata', 'repavg', 'stimulus_meta', 'trials']


In [4]:
!ls {dataroot}

nsd_shared1000_6monkeys_2024.h5
paul_20250428-20250430_vvs-encodingstimuli_z1_rw100-400.h5
paul_denoised.npy
red_20250428-20240429_vvs-encodingstimuli_z1_rw100-400.h5
red_20250428-20250430_vvs-encodingstimuli_z1_rw100-400.h5
three0_250426-250430_vvs-encodingstimuli_z1_rw80-250.h5
venus_250426-250429_vvs-encodingstimuli_z1_rw80-250.h5
vvs-accentuate-day1_normalize_paul_241119-241122.h5
vvs-accentuate-day1_normalize_paul_241119.h5
vvs-accentuate-day1_normalize_red_20241212-20241220.h5
vvs-accentuate-day1_raw_paul_241119-241122.h5
vvs-accentuate-day2_normalize_paul_241120.h5
vvs-accentuate-day3_normalize_paul_241124-25.h5
vvs_accentuate_day3_normalize_red_20250123-20250126.hdf5


In [5]:
for filename in [
    "red_20250428-20250430_vvs-encodingstimuli_z1_rw100-400.h5", 
    "paul_20250428-20250430_vvs-encodingstimuli_z1_rw100-400.h5", 
    "three0_250426-250430_vvs-encodingstimuli_z1_rw80-250.h5", 
    "venus_250426-250429_vvs-encodingstimuli_z1_rw80-250.h5", 
]:
    data_path = join(dataroot, filename)
    data = load_from_hdf5(data_path)
    print(list(data))

['README', 'neuron_metadata', 'repavg', 'stimulus_meta', 'trials']
['README', 'neuron_metadata', 'repavg', 'stimulus_meta', 'trials']
['README', 'neuron_metadata', 'repavg', 'stimulus_meta', 'trials']
['README', 'neuron_metadata', 'repavg', 'stimulus_meta', 'trials']


In [6]:
data["neuron_metadata"]

{'brain_area': {'neuropixel': array([b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS',
         b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS',
         b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS',
         b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS',
         b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS',
         b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS',
         b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS',
         b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS',
         b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS',
         b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS',
         b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS',
         b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS',
         b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS',
         b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS', b'l_STS',
      

In [8]:
data["README"]

{'neuron_metadata': {'brain_area': 'Brain area / array location.',
  'ncsnr': 'Noise‑ceiling SNR (NSD metric).',
  'peak_respwindow': 'Window used for peak response averaging.',
  'reliability': 'Split‑half reliability of each neuron.',
  'temporal_windows': 'Center of each 10\u202fms time bin.'},
 'repavg': {'response_peak': 'Repeat‑averaged peak response (n_stims, n_neurons).',
  'response_temporal': 'Repeat‑averaged temporal response (n_timepoints, n_stims, n_neurons).',
  'stimulus_name': 'Unique stimulus file names.'},
 'trials': {'response_peak': 'Response of each neuron during the peak window, shape (n_trials, n_neurons).',
  'response_temporal': 'Time‑series response, shape (n_timepoints, n_trials, n_neurons).',
  'session_num': 'Integer session label for each trial.',
  'stimulus_name': 'File name shown on each trial.',
  'stimulus_pos_deg': 'XY position (deg) on each trial.',
  'stimulus_size_pix': 'H×W pixels on each trial.'}}

In [15]:
def extract_neural_data_dict_2025apr(data):
    data_dict = {}
    data_dict['resp_mat'] = data['repavg']["response_peak"]
    data_dict['resp_temp_mat'] = data['repavg']["response_temporal"]
    data_dict['reliability'] = data['neuron_metadata']["reliability"]
    data_dict['ncsnr'] = data['neuron_metadata']["ncsnr"]
    data_dict['brain_area'] = data['neuron_metadata']["brain_area"]
    data_dict['stim_pos'] = data['stimulus_meta']['xy_deg']
    data_dict['stim_size'] = data["stimulus_meta"]["size_px"]
    data_dict['stim_size'] = data["stimulus_meta"]["size_px"]
    data_dict["stimulus_names"] = data['repavg']['stimulus_name']
    return data_dict

encoding_stim_dir = r"/n/holylabs/LABS/alvarez_lab/Lab/VVS_Accentuation/Stimuli/encodingstimuli_apr2025"
for subject_id, filename in [
    ("red_20250428-20250430", "red_20250428-20250430_vvs-encodingstimuli_z1_rw100-400.h5"), 
    ("paul_20250428-20250430", "paul_20250428-20250430_vvs-encodingstimuli_z1_rw100-400.h5"), 
    ("three0_250426-250430", "three0_250426-250430_vvs-encodingstimuli_z1_rw80-250.h5"), 
    ("venus_250426-250429", "venus_250426-250429_vvs-encodingstimuli_z1_rw80-250.h5"), 
]:
    print(subject_id)
    data_path = join(dataroot, filename)
    data = load_from_hdf5(data_path)
    print(list(data))
    data_dict = extract_neural_data_dict_2025apr(data)
    data_dict['image_fps'] = parse_image_fullpaths(data_dict["stimulus_names"], [encoding_stim_dir], arbitrary_format=True)
    image_fps = data_dict['image_fps']
    resp_mat = data_dict['resp_mat']
    reliability = data_dict['reliability']
    ncsnr = data_dict['ncsnr']

red_20250428-20250430
['README', 'neuron_metadata', 'repavg', 'stimulus_meta', 'trials']
All stimulus files were found
paul_20250428-20250430
['README', 'neuron_metadata', 'repavg', 'stimulus_meta', 'trials']
All stimulus files were found
three0_250426-250430
['README', 'neuron_metadata', 'repavg', 'stimulus_meta', 'trials']
All stimulus files were found
venus_250426-250429
['README', 'neuron_metadata', 'repavg', 'stimulus_meta', 'trials']
All stimulus files were found
